<a href="https://colab.research.google.com/github/gowravmannem/HackAi_2024/blob/main/Bruteforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score


df = pd.read_csv('Housing.csv')
df

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

df['mainroad'] = le.fit_transform(df['mainroad'])
df['guestroom'] = le.fit_transform(df['guestroom'])
df['basement'] = le.fit_transform(df['basement'])
df['hotwaterheating'] = le.fit_transform(df['hotwaterheating'])
df['airconditioning'] = le.fit_transform(df['airconditioning'])
df['prefarea'] = le.fit_transform(df['prefarea'])

df

In [ ]:
dummies = pd.get_dummies(df.furnishingstatus)

# Concatenate the dummies to original dataframe
merged = pd.concat([df, dummies], axis='columns')

# drop the values
df_new = merged.drop(columns=['furnishingstatus'])

# print the dataframe
df_new

In [ ]:
# taking out Area data outliers
df1 = df_new.copy()

#features1 = [i for i in features if i not in ['CHAS','RAD']]
features1 = ["area"]

for i in features1:
    Q1 = df1[i].quantile(0.25)
    Q3 = df1[i].quantile(0.75)
    IQR = Q3 - Q1
    df1 = df1[df1[i] <= (Q3+(1.5*IQR))]
    df1 = df1[df1[i] >= (Q1-(1.5*IQR))]
    df1 = df1.reset_index(drop=True)
df1

In [ ]:
X = df1.drop(['price'],axis=1)
y = df1['price']


In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)

checkVIF(X)

In [ ]:
#solving multicollinearity issue
X=X.drop(["furnished","semi-furnished","unfurnished"],axis=1)

In [ ]:

X_train,X_test,Train_Y,Test_Y = train_test_split(X,y,train_size=0.85, test_size=0.15, random_state=100)

In [ ]:
import pandas as pd
#scaler = MinMaxScaler()
#standard scaler works better
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
Train_X_std = pd.DataFrame(X_train_scaled, columns=X_train.columns)

X_test_scaled = scaler.fit_transform(X_test)
Test_X_std = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
Model_Evaluation_Comparison_Matrix = pd.DataFrame(np.zeros([5,8]), columns=['Train-R2','Test-R2','Train-RSS','Test-RSS','Train-MSE','Test-MSE','Train-RMSE','Test-RMSE'])
rc=np.random.choice(Train_X_std.loc[:,Train_X_std.nunique()>=50].columns.values,1,replace=False)
def Evaluate(n, pred1,pred2):
    #Plotting predicted predicteds alongside the actual datapoints
    plt.figure(figsize=[15,6])
    for e,i in enumerate(rc):
        plt.subplot(2,3,e+1)
        plt.scatter(y=Train_Y, x=Train_X_std[i], label='Actual')
        plt.scatter(y=pred1, x=Train_X_std[i], label='Prediction')
        plt.legend()
    plt.show()

    #Evaluating the Multiple Linear Regression Model

    print('\n\n{}Training Set Metrics{}'.format('-'*20, '-'*20))
    print('\nR2-Score on Training set --->',round(r2_score(Train_Y, pred1),20))
    print('Mean Squared Error (MSE) on Training set       --->',round(mean_squared_error(Train_Y, pred1),20))
    print('Root Mean Squared Error (RMSE) on Training set --->',round(np.sqrt(mean_squared_error(Train_Y, pred1)),20))

    print('\n{}Testing Set Metrics{}'.format('-'*20, '-'*20))
    print('\nR2-Score on Testing set --->',round(r2_score(Test_Y, pred2),20))
    print('Mean Squared Error (MSE) on Training set       --->',round(mean_squared_error(Test_Y, pred2),20))
    print('Root Mean Squared Error (RMSE) on Training set --->',round(np.sqrt(mean_squared_error(Test_Y, pred2)),20))
    print('\n{}Residual Plots{}'.format('-'*20, '-'*20))

    Model_Evaluation_Comparison_Matrix.loc[n,'Train-R2']  = round(r2_score(Train_Y, pred1),20)
    Model_Evaluation_Comparison_Matrix.loc[n,'Test-R2']   = round(r2_score(Test_Y, pred2),20)
    Model_Evaluation_Comparison_Matrix.loc[n,'Train-MSE'] = round(mean_squared_error(Train_Y, pred1),20)
    Model_Evaluation_Comparison_Matrix.loc[n,'Test-MSE']  = round(mean_squared_error(Test_Y, pred2),20)
    Model_Evaluation_Comparison_Matrix.loc[n,'Train-RMSE']= round(np.sqrt(mean_squared_error(Train_Y, pred1)),20)
    Model_Evaluation_Comparison_Matrix.loc[n,'Test-RMSE'] = round(np.sqrt(mean_squared_error(Test_Y, pred2)),20)

    # Plotting y_test and y_pred to understand the spread.
    plt.figure(figsize=[15,4])

    plt.subplot(1,2,1)
    sns.distplot((Train_Y - pred1))
    plt.title('Error Terms')
    plt.xlabel('Errors')

    plt.subplot(1,2,2)
    plt.scatter(Train_Y,pred1)
    plt.plot([Train_Y.min(),Train_Y.max()],[Train_Y.min(),Train_Y.max()], 'r--')
    plt.title('Test vs Prediction')
    plt.xlabel('y_test')
    plt.ylabel('y_pred')
    plt.show()

In [ ]:
RFR = RandomForestRegressor().fit(Train_X_std,Train_Y)
pred1 = RFR.predict(Train_X_std)
pred2 = RFR.predict(Test_X_std)

Evaluate(0, pred1, pred2)

In [ ]:
RLR = Ridge().fit(Train_X_std,Train_Y)
pred1 = RLR.predict(Train_X_std)
pred2 = RLR.predict(Test_X_std)

Evaluate(1, pred1, pred2)

In [ ]:
LLR = Lasso().fit(Train_X_std,Train_Y)
pred1 = LLR.predict(Train_X_std)
pred2 = LLR.predict(Test_X_std)

Evaluate(2, pred1, pred2)

In [ ]:
ENR = ElasticNet().fit(Train_X_std,Train_Y)
pred1 = ENR.predict(Train_X_std)
pred2 = ENR.predict(Test_X_std)


Evaluate(3, pred1, pred2)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor().fit(Train_X_std,Train_Y)
pred1 = GBR.predict(Train_X_std)
pred2 = GBR.predict(Test_X_std)

Evaluate(4, pred1, pred2)

In [ ]:
XGB = xgb.XGBRegressor().fit(Train_X_std,Train_Y)
pred1 = XGB.predict(Train_X_std)
pred2 = XGB.predict(Test_X_std)

Evaluate(5, pred1, pred2)

In [ ]:
XGRF = xgb.XGBRFRegressor().fit(Train_X_std,Train_Y)
pred1 = XGRF.predict(Train_X_std)
pred2 = XGRF.predict(Test_X_std)

Evaluate(6, pred1, pred2)

In [ ]:
LGBM = LGBMRegressor().fit(Train_X_std,Train_Y)
pred1 = LGBM.predict(Train_X_std)
pred2 = LGBM.predict(Test_X_std)

Evaluate(7, pred1, pred2)

In [ ]:
CBR = CatBoostRegressor().fit(Train_X_std,Train_Y)
pred1 = CBR.predict(Train_X_std)
pred2 = CBR.predict(Test_X_std)

Evaluate(8, pred1, pred2)

In [ ]:
EMC = Model_Evaluation_Comparison_Matrix.copy()
EMC.index = ['Random Forest Regressor (RFR)','Ridge Linear Regression (RLR)','Lasso Linear Regression (LLR)','Elastic-Net Regression (ENR)','Gradient Boost Regressor (GBR)','XGB',"XGRF","LGBM","CBR"]
EMC